Naar: https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a

In [1]:
# Deze cel hoeft alleen als MacOS klaagt over Certificates
# hieronder bij het downloaden van de dataset maar kan
# anders vast ook geen kwaad... ;-)
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
# Download en lees de dataset
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', shuffle=True)
twenty_test = fetch_20newsgroups(subset='test', shuffle=True)

In [3]:
# Print de klasselabels
twenty_train.target_names #prints all the categories

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [4]:
# Hoeveel trainingsinstances zijn er?
print(len(twenty_train.data))

11314


In [5]:
# Print a random instance
from random import randint
print(twenty_train.data[randint(0, len(twenty_train.data)-1)])

From: ktj@beach.cis.ufl.edu (kerry todd johnson)
Subject: army in space
Organization: Univ. of Florida CIS Dept.
Lines: 17
Distribution: world
NNTP-Posting-Host: beach.cis.ufl.edu


Is anybody out there willing to discuss with me careers in the Army that deal
with space?  After I graduate, I will have a commitment to serve in the Army, 
and I would like to spend it in a space-related field.  I saw a post a long
time ago about the Air Force Space Command which made a fleeting reference to
its Army counter-part.  Any more info on that would be appreciated.  I'm 
looking for things like: do I branch Intelligence, or Signal, or other?  To
whom do I voice my interest in space?  What qualifications are necessary?
Etc, etc.  BTW, my major is computer science engineering.

Please reply to ktj@reef.cis.ufl.edu

Thanks for ANY info.
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
= Whether they ever find life there or not, I think Jupiter should be         =
= con

In [6]:
# Bepaal welke woorden hoe vaak voorkomen in elke instance
# Dit heet het "Bag of Words" model
# Negeert de niet-betekenisvolle "stopwoorden" en zeldzame
# woorden
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(stop_words='english', max_features=10000, max_df=0.9)
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(11314, 10000)

<div class="alert alert-block alert-info">Word Stemming/Lemmatization wordt ook gedaan, betreft het samenvoegen van verschillende uitgangen (bv. meervouden, werkwoordsvervoegingen, e.d.), maar doe ik even niet omdat dit over EN tekst gaat en wij NL gaan doen.</div>

In [7]:
# Of beter, zet aantallen om in frequenties
# Term Frequency times Inverse Document Frequency
# deelt aantallen zowel door totale aantal per
# instance en door log-totale aantal per woord
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(11314, 10000)

In [8]:
# Train bv. NB
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

In [9]:
# Dit kan ook in één keer met een pipeline
from sklearn.pipeline import Pipeline
import numpy as np
text_nb = Pipeline([('vect', CountVectorizer(stop_words='english', max_features=10000, max_df=0.9)),
                    ('tfidf', TfidfTransformer()),
                    ('nb', MultinomialNB()),
])
text_nb = text_nb.fit(twenty_train.data, twenty_train.target)
predicted_nb = text_nb.predict(twenty_test.data)
np.mean(predicted_nb == twenty_test.target)

0.8086829527349974

In [10]:
# Idem met Support Vector Machine
from sklearn.linear_model import SGDClassifier
text_svm = Pipeline([('vect', CountVectorizer(stop_words='english', max_features=10000, max_df=0.9)),
                     ('tfidf', TfidfTransformer()),
                     ('svm', SGDClassifier()),
])
text_svm.fit(twenty_train.data, twenty_train.target)
predicted_svm = text_svm.predict(twenty_test.data)
np.mean(predicted_svm == twenty_test.target)

0.8271375464684015

Hierna kijken naar Word Embeddings i.p.v. Bag of Words. Zie bv.:

* https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/

* https://realpython.com/python-keras-text-classification/

* https://medium.com/analytics-vidhya/text-classification-using-word-embeddings-and-deep-learning-in-python-classifying-tweets-from-6fe644fcfc81

* https://stackabuse.com/python-for-nlp-word-embeddings-for-deep-learning-in-keras/

***